# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from task_three import task_three
from utils import general_utils, data_prep_utils, feature_selection_utils


In [2]:
tcga_genes = data_prep_utils.gene_exp_log_trans(data_prep_utils.get_df("tcga_rna", nb=True))
tcga_muts = data_prep_utils.get_df("tcga_mut", nb=True)
tcga_muts_int = data_prep_utils.mut_df_label_to_int(tcga_muts)

In [3]:
tcga_muts_int.head()

,DNMT3A,IDH1,NPM1,IDH2,RUNX1,NRAS,CEBPA,FLT3,TP53,TET2,WT1
TCGA.AB.2803,0,0,0,0,0,0,0,0,0,0,0
TCGA.AB.2805,0,0,0,1,1,0,0,0,0,0,0
TCGA.AB.2806,0,0,0,0,0,0,0,0,0,0,0
TCGA.AB.2807,0,0,0,1,1,0,0,0,0,0,0
TCGA.AB.2808,0,0,0,0,0,1,1,0,0,0,0


# Test build_cor_mat

In [5]:
drugs_dict = {"d1": [0, 1, 2], "d2": [3, 4, 5]}
mut_dict = {"mut1": [1, 2, 3], "mut2": [3, 4, 5], "mut3": [18, 2, 97]}

drugs_df = pd.DataFrame.from_dict(drugs_dict)
print(drugs_df)
mut_df = pd.DataFrame.from_dict(mut_dict)
print(mut_df)

cor_mat = task_three.build_cor_mat(drugs_df, mut_df)
print(cor_mat)
testings_utils.export_drugs_prediction(cor_mat, "test_cor_mat")

   d1  d2
0   0   3
1   1   4
2   2   5
   mut1  mut2  mut3
0     1     3    18
1     2     4     2
2     3     5    97
            d1        d2
mut1  1.000000  1.000000
mut2  1.000000  1.000000
mut3  0.776603  0.776603


# Test get_linear_svc_model

In [3]:
x_train, x_test, y_train, y_test = train_test_split(tcga_genes, tcga_muts, test_size = 0.2, random_state = 42)

In [7]:
svc_model = task_three.get_linear_svc_model(tcga_genes, tcga_muts)

C:\Users\yaelz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


KeyboardInterrupt: 

# Test get_gradient_boosting_class_model

In [6]:
gbc = task_three.get_gradient_boosting_class_model(x_train, y_train)

KeyboardInterrupt: 

# Test KNN neighbors

### feature selections

In [5]:
tcga_genes, filtered_genes = feature_selection_utils.select_gene_by_corrlation(tcga_genes, tcga_muts)

1  -  TCGA.AB.2803  ->  0
2  -  TCGA.AB.2805  ->  0
3  -  TCGA.AB.2806  ->  0
4  -  TCGA.AB.2807  ->  0
5  -  TCGA.AB.2808  ->  0
6  -  TCGA.AB.2810  ->  0
7  -  TCGA.AB.2811  ->  0
8  -  TCGA.AB.2812  ->  0
9  -  TCGA.AB.2813  ->  0
10  -  TCGA.AB.2814  ->  0
11  -  TCGA.AB.2816  ->  0
12  -  TCGA.AB.2817  ->  0
13  -  TCGA.AB.2818  ->  0
14  -  TCGA.AB.2819  ->  0
15  -  TCGA.AB.2820  ->  0
16  -  TCGA.AB.2821  ->  0
17  -  TCGA.AB.2822  ->  0
18  -  TCGA.AB.2823  ->  0
19  -  TCGA.AB.2824  ->  0
20  -  TCGA.AB.2825  ->  0
21  -  TCGA.AB.2826  ->  0
22  -  TCGA.AB.2828  ->  0
23  -  TCGA.AB.2830  ->  0
24  -  TCGA.AB.2832  ->  0
25  -  TCGA.AB.2833  ->  0
26  -  TCGA.AB.2834  ->  0
27  -  TCGA.AB.2835  ->  0
28  -  TCGA.AB.2836  ->  0
29  -  TCGA.AB.2837  ->  0
30  -  TCGA.AB.2838  ->  0
31  -  TCGA.AB.2839  ->  0
32  -  TCGA.AB.2840  ->  0
33  -  TCGA.AB.2841  ->  0
34  -  TCGA.AB.2842  ->  0
35  -  TCGA.AB.2843  ->  0
36  -  TCGA.AB.2844  ->  0
37  -  TCGA.AB.2845  ->  0
38  -  TCG